In [ ]:
import pre
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [ ]:
layer = 1

In [ ]:
label = pre.get_label(layer)
corpus = pre.create_sentence_list(1)
from nltk.stem import SnowballStemmer 

st = SnowballStemmer("english")
text = pre.norm_corpus

output = []
for sentence in text:
    output.append(' '.join([st.stem(word) for word in sentence.split()]))
fileid=pre.fileid
tv = TfidfVectorizer(min_df=0., max_df=1., max_features=1500, use_idf=False, lowercase=True, ngram_range=(1,2), analyzer=u'word', smooth_idf=False)
tv_matrix = tv.fit_transform(output)
tv_matrix = tv_matrix.toarray()

print(len(tv_matrix))
print(label)
print(pre.norm_corpus)

In [ ]:
new_label = []
new_tv = []
new_fileid = []
new_corpus = []
length = len(tv_matrix)
for i in range(length):
    for j in range(len(label[i])):
        new_label.append(label[i][j])
        new_tv.append(tv_matrix[i])
        new_fileid.append(fileid[i])
        new_corpus.append(corpus[i])

In [ ]:
import numpy as np
num_rows = len(new_tv)
num_cols = len(new_tv[0])
modified_tv = []
padding = np.zeros(num_cols, dtype=int)

for i in range(num_rows):
    if i == 0:
        x = np.append(padding, new_tv[0])
        x = np.append(x, new_tv[1])
        modified_tv.append(x)
        
    elif i == (num_rows - 1):
        x = np.append(new_tv[num_rows - 2], new_tv[num_rows - 1])
        x = np.append(x, padding)
        modified_tv.append(x)
        
    else: 
        x = np.append(new_tv[i - 1], new_tv[i])
        x = np.append(x, new_tv[i + 1])
        modified_tv.append(x)
        
df_tfidf = pre.pd.DataFrame(pre.np.round(modified_tv, 10))
df_tfidf['tag'] = new_label

In [ ]:
import numpy as np

X_df = df_tfidf[df_tfidf.columns[:-1]]
X = X_df.to_numpy()
y = df_tfidf['tag']
print(np.unique(df_tfidf['tag']))
class_names = np.unique(df_tfidf['tag'])

In [ ]:
from sklearn.model_selection import train_test_split
from collections import Counter
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, shuffle = False)
y_train = y_train.reset_index()['tag'].tolist()
print(X_train.shape)
print(len(y_train))
print(length)
print(y_train)

Logistic 

In [ ]:
import sklearn.metrics as metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report

logmodel = LogisticRegression(class_weight='balanced', solver='liblinear')
logmodel.fit(X_train, y_train)
y_pred = logmodel.predict(X_test)

print(classification_report(y_test, y_pred, zero_division=1))
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix

np.set_printoptions(precision=3)

titles_options = [("Confusion matrix without normalization", None),
                  ("Normlaized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(logmodel, X_test, y_test,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)
    
def heatconmat(y_true, y_pred):
    sns.set_context('talk')
    plt.figure(figsize=(10,8))
    sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, fmt='d', cbar=False, cmap='gist_earth_r', yticklabels=sorted(y_test.unique()))
    plt.show()
    print(classification_report(y_true, y_pred, zero_division=1))

Multinomial 

In [ ]:
import sklearn.metrics as metrics
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, zero_division=1))
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix

np.set_printoptions(precision=3)

titles_options = [("Confusion matrix without normalization", None),
                  ("Normlaized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(clf, X_test, y_test,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

plt.show()

Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics

clf = RandomForestClassifier(n_estimators=100, bootstrap=True, max_features='sqrt', class_weight='balanced_subsample')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, zero_division=1))
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix

np.set_printoptions(precision=3)

titles_options = [("Confusion matrix without normalization", None),
                    ("Normlaized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(clf, X_test, y_test,
                                    display_labels=class_names,
                                    cmap=plt.cm.Blues,
                                    normalize=normalize)
    disp.ax_.set_title(title)
plt.show()

SVM

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split

clf = svm.SVC(class_weight='balanced', decision_function_shape='ovo')
import sklearn.metrics as metrics
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, zero_division=1))
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
np.set_printoptions(precision=3)

titles_options = [("Confusion matrix without normalization", None),
                    ("Normlaized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(clf, X_test, y_test,
                                    display_labels=class_names,
                                    cmap=plt.cm.Blues,
                                    normalize=normalize)
    disp.ax_.set_title(title)
plt.show()

SGD

In [ ]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='hinge', penalty='12', max_iter=5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, zero_division=1))
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
np.set_printoptions(precision=3)

titles_options = [("Confusion matrix without normalization", None),
                    ("Normlaized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(clf, X_test, y_test,
                                    display_labels=class_names,
                                    cmap=plt.cm.Blues,
                                    normalize=normalize)
    disp.ax_.set_title(title)
plt.show()